In [17]:
from functools import lru_cache

@lru_cache
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a mapping to unicode strings. We specifically avoids mapping to whitespace/control
    characters the bpe code barfs on.

    The reversible bpe codes work on unicode strings. This means you need a large # of unicode characters in your vocab
    if you want to avoid UNKs. When you're at something like a 10B token dataset you end up needing around 5K for
    decent coverage. This is a significant percentage of your normal, say, 32K bpe vocab. To avoid that, we want lookup
    tables between utf-8 bytes and unicode strings.
    """
    bs = (
        list(range(ord("!"), ord("~") + 1)) + list(range(ord("¡"), ord("¬") + 1)) + list(range(ord("®"), ord("ÿ") + 1))
    )
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8 + n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))

byte_encoder = bytes_to_unicode()
len(byte_encoder), byte_encoder.get(47)

(256, '/')

In [28]:
import regex as re
PRETOKENIZE_REGEX = r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"""
tokens = re.findall(re.compile(PRETOKENIZE_REGEX), "It supposed to be a dragon. That's cool!! ... Ġsupposedasdffadsgb")
tokens

['It',
 ' supposed',
 ' to',
 ' be',
 ' a',
 ' dragon',
 '.',
 ' That',
 "'s",
 ' cool',
 '!!',
 ' ...',
 ' Ġsupposedasdffadsgb']

In [20]:
def get_pairs(word):
    """
    Return set of symbol pairs in a word.

    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

get_pairs('supposed')

{('e', 'd'),
 ('o', 's'),
 ('p', 'o'),
 ('p', 'p'),
 ('s', 'e'),
 ('s', 'u'),
 ('u', 'p')}

In [21]:
bpe_merges = []

with open("./merges.txt", encoding="utf-8") as merges_handle:
    for i, line in enumerate(merges_handle):
        line = line.strip()
        if (i == 0 and line.startswith("#version:")) or not line:
            continue
        bpe_merges.append(tuple(line.split()))
        
bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))
len(bpe_ranks), bpe_ranks.get(('l', 'o'))

(151387, 129)

In [29]:
byte_tokens = []
for token in tokens:
    token = "".join(byte_encoder[b] for b in token.encode("utf-8"))
    byte_tokens.append(token)

byte_tokens

['It',
 'Ġsupposed',
 'Ġto',
 'Ġbe',
 'Ġa',
 'Ġdragon',
 '.',
 'ĠThat',
 "'s",
 'Ġcool',
 '!!',
 'Ġ...',
 'ĠÄłsupposedasdffadsgb']

In [30]:
@lru_cache
def bpe(token):  
    word = tuple(token)
    pairs = get_pairs(word)

    if not pairs:
        return token

    while True:
        bigram = min(pairs, key=lambda pair: bpe_ranks.get(pair, float("inf")))
        if bigram not in bpe_ranks:
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
            except ValueError:
                new_word.extend(word[i:])
                break
            else:
                new_word.extend(word[i:j])
                i = j

            if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)
            
    word = " ".join(word)
    return word

bpe("Ġsupposedasdffadsgb")

'Ġsupposed asd ff ad sg b'

In [31]:
bpe_tokens = []
for token in byte_tokens:
    bpe_tokens.extend(bpe_token for bpe_token in bpe(token).split(" "))

bpe_tokens

['It',
 'Ġsupposed',
 'Ġto',
 'Ġbe',
 'Ġa',
 'Ġdragon',
 '.',
 'ĠThat',
 "'s",
 'Ġcool',
 '!!',
 'Ġ...',
 'ĠÄ',
 'ł',
 'sup',
 'posed',
 'asd',
 'ff',
 'ad',
 'sg',
 'b']